### Follows Ulissi's group 
https://github.com/ulissigroup/uncertainty_benchmarking/blob/master/preprocessing/split_data.ipynb

In [1]:
import pickle
import numpy as np 

In [2]:
with open('/scratch/westgroup/mpnn/gasdb_dgl_graphs/gasdb_dgl_graphs.pkl','rb') as infile:
    graphs = pickle.load(infile)

In [3]:
graphs = [graph for graph in graphs if graph is not np.nan]

In [4]:
len(graphs)

47279

In [5]:
import random
from sklearn.model_selection import train_test_split

In [6]:
data_train, data_remain = [], [] 
for graph in graphs:
    if len(graph.comp)-1 == 1 ## mono metallics
        data_train.append(graph)
    else:
        data_remain.append(graph)

In [9]:
adsorbates =[graph.adsorbate for graph in graphs]

In [11]:
adsorbates = list(set(adsorbates))

In [13]:
# Stratify our data by adsorbate
data_val = []
data_test = []
for ads in adsorbates:
    _data = [graph
             for graph in data_remain
             if graph.adsorbate == ads]

    # Split out the testing and validation data
    data_cv, _data_test = train_test_split(_data, test_size=0.2)
    _data_train, _data_val = train_test_split(data_cv, test_size=0.2)

    # Concatenate the data in this split with the rest
    data_train.extend(_data_train)
    data_val.extend(_data_val)
    data_test.extend(_data_test)

In [14]:
# Shuffle all the datasets because they've been sorted by both adsorbate and monometallics
random.shuffle(data_train)
random.shuffle(data_val)
random.shuffle(data_test)

In [15]:
len(data_train), len(data_val), len(data_test)

(31466, 7028, 8785)

In [16]:
# Report the final splits
print('%i%% train' % round(len(data_train)/len(graphs) * 100))
print('%i%% validate' % round(len(data_val)/len(graphs) * 100))
print('%i%% test' % round(len(data_test)/len(graphs) * 100))

67% train
15% validate
19% test


In [17]:
with open('/scratch/westgroup/mpnn/gasdb_dgl_graphs/gaspy_splits/gaspy_train.pkl','wb') as outfile:
    pickle.dump(data_train, outfile)

In [19]:
with open('/scratch/westgroup/mpnn/gasdb_dgl_graphs/gaspy_splits/gaspy_valid.pkl','wb') as outfile:
    pickle.dump(data_val, outfile)

In [20]:
with open('/scratch/westgroup/mpnn/gasdb_dgl_graphs/gaspy_splits/gaspy_test.pkl','wb') as outfile:
    pickle.dump(data_test, outfile)